In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if 'train' in filename:
            data = pd.read_csv(os.path.join(dirname, filename))
            print('Train readed')
        if 'test' in filename:
            test_data = pd.read_csv(os.path.join(dirname, filename))
            print('Tdataest readed')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data['Age'].fillna(data.Age.mean(), inplace=True)
data = pd.get_dummies(data,columns=['Sex','Embarked'],drop_first=True)
test_data['Age'].fillna(test_data.Age.mean(), inplace=True)
test_data = pd.get_dummies(test_data,columns=['Sex','Embarked'],drop_first=True)

labels = data['Survived'].copy()
train_data = data.drop(labels=['Name','Survived','PassengerId','Ticket','Cabin'], axis=1)
test_data = test_data.drop(labels=['Name','PassengerId','Ticket','Cabin'], axis=1)

data.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler().fit(train_data)
X_train_std = sc.transform(train_data)
X_test_std = sc.transform(test_data)


In [ ]:
corr_matrix = data.corr() #coeficiente de correlacion Pearson's r
corr_matrix["Survived"].sort_values(ascending=False)

In [ ]:
train_data.head()


In [ ]:
train_data = train_data.drop(labels=['Parch','Embarked_Q','SibSp','Age'], axis=1)
test_data = test_data.drop(labels=['Parch','Embarked_Q','SibSp','Age'], axis=1)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
params = {'C': [0.001,0.01,0.1,1,10,0.005,0.05,0.5,5],
            'kernel': ['poly','rbf','linear'],
            'gamma':[0.5,0.1,0.001]} 

svm_grid = GridSearchCV(SVC(), params, n_jobs=-1, cv=10,verbose=1, scoring='accuracy')
svm_grid.fit(X_train_std,labels)

In [ ]:
from sklearn.metrics import accuracy_score
svm_model = svm_grid.best_estimator_
y_predicted_svm = svm_model.predict(X_train_std)
accuracy_score(labels,y_predicted_svm)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
params = {'n_neighbors': [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
knn_grid = GridSearchCV(KNeighborsClassifier(),params, n_jobs=-1, cv=10,verbose=1, scoring='accuracy')
grid_search_cv.fit(X_train_std,labels)

In [ ]:
knn_model = knn_grid.best_estimator_
y_predicted_knn = knn_model.predict(X_train_std)

accuracy_score(labels,y_predicted_knn)

In [ ]:

test = pd.DataFrame(X, columns=test.columns)
kn_survival_predict = kn_model.predict(test)


In [ ]:
submission = pd.DataFrame({'PassengerId':test_data['PassengerId'],'Survived':kn_survival_predict})
submission.head()

In [ ]:
filename = 'Predictions.csv'
submission.to_csv(filename,index=False)